In [4]:
# class

import torch
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

# prepare data
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))]) # 转换为网络需要的格式以及标准化，均值，方差

train_dataset = datasets.MNIST(root='data/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_dataset = datasets.MNIST(root='data/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)


Using downloaded and verified file: data/mnist/MNIST\raw\train-images-idx3-ubyte.gz
Extracting data/mnist/MNIST\raw\train-images-idx3-ubyte.gz to data/mnist/MNIST\raw


113.5%

Extracting data/mnist/MNIST\raw\train-labels-idx1-ubyte.gz to data/mnist/MNIST\raw


100.4%

Extracting data/mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to data/mnist/MNIST\raw


180.4%D:\Anaconda\envs\pytorch_gpu\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting data/mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/mnist/MNIST\raw
Processing...
Done!


In [6]:
# design model using class

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(784, 512)
        self.l2 = torch.nn.Linear(512, 256)
        self.l3 = torch.nn.Linear(256, 128)
        self.l4 = torch.nn.Linear(128, 64)
        self.l5 = torch.nn.Linear(64, 10)
    
    def forward(self, x):
        x = x.view(-1, 784)  # -1表示自动计算784的个数，也就是有多少个样本
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) # 最后一层不做激活，因为要用crossentropy，自带了softmax
    
model = Net()

In [7]:
# construct loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.5)

# train cycle
def train(epoch):
    running_loss = 0.0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        optimizer.zero_grad()
        # 获得预测结果
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if batch_idx % 300 == 299:
            print("[%d, %5d] loss: %.3f" % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0

def test():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _,predicted = torch.max(outputs.data, dim=1) #dim=1，从列的方向找寻最大值，返回的第一个参数是值，第二个值是下标
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print("accracy on test_set: %d %%" % (100*correct/total))

In [8]:
for epoch in range(10):
    train(epoch)
    test()

[1,   300] loss: 2.248
[1,   600] loss: 1.043
[1,   900] loss: 0.457
accracy on test_set: 87 %
[2,   300] loss: 0.327
[2,   600] loss: 0.272
[2,   900] loss: 0.243
accracy on test_set: 94 %
[3,   300] loss: 0.190
[3,   600] loss: 0.173
[3,   900] loss: 0.151
accracy on test_set: 95 %
[4,   300] loss: 0.132
[4,   600] loss: 0.117
[4,   900] loss: 0.121
accracy on test_set: 96 %
[5,   300] loss: 0.099
[5,   600] loss: 0.097
[5,   900] loss: 0.089
accracy on test_set: 96 %
[6,   300] loss: 0.074
[6,   600] loss: 0.080
[6,   900] loss: 0.073
accracy on test_set: 97 %
[7,   300] loss: 0.063
[7,   600] loss: 0.061
[7,   900] loss: 0.061
accracy on test_set: 97 %
[8,   300] loss: 0.049
[8,   600] loss: 0.047
[8,   900] loss: 0.055
accracy on test_set: 97 %
[9,   300] loss: 0.038
[9,   600] loss: 0.044
[9,   900] loss: 0.044
accracy on test_set: 97 %
[10,   300] loss: 0.031
[10,   600] loss: 0.035
[10,   900] loss: 0.034
accracy on test_set: 97 %


# practice-otto

In [4]:
# otto
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

# data_train = pd.read_csv("data/otto/train.csv")
# data_test = pd.read_csv("data/otto/test.csv")

# 将类别标签转换为数字
def label2id(lables):
    target_id = []
    target_labels = ['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
    for label in lables:
        target_id.append(target_labels.index(label))
    return target_id

batch_size = 64

# prepare dataset
class OttoDataset(Dataset):
    def __init__(self, filepath):
        data = pd.read_csv(filepath)
        lables = data['target']
        self.len = data.shape[0]
        
        self.x_data = torch.from_numpy(np.array(data)[:,1:-1].astype(float)) #必须转换成float
        self.y_data = label2id(lables)
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.len

data_train = OttoDataset("data/otto/train.csv")
# data_test = OttoDataset("data/otto/test.csv")
train_loader = DataLoader(dataset=data_train, batch_size=batch_size, shuffle=True)


In [16]:
# design model using class

class ottoNet(torch.nn.Module):
    def __init__(self):
        super(ottoNet, self).__init__()
        self.l1 = torch.nn.Linear(93, 64)
        self.l2 = torch.nn.Linear(64, 32)
        self.l3 = torch.nn.Linear(32, 16)
        self.l4 = torch.nn.Linear(16, 9)
    
    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        return self.l4(x) # 最后一层不做激活，因为要用crossentropy，自带了softmax
    
    def predict(self, x):
        with torch.no_grad():
            x = F.relu(self.l1(x))
            x = F.relu(self.l2(x))
            x = F.relu(self.l3(x))
            x = self.l4(x)
            _, predicted = torch.max(x, dim=1)
            # 将预测的类别转为one-hot表示，方便保存为预测文件。
#             y = pd.get_dummies(predicted)
            return predicted
            
model = ottoNet()

In [10]:
# contruct loss and optimzer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

def train(epoch):
    running_loss=0.0
    for batch_idx, data in enumerate(train_loader):
        inputs, target = data
        inputs = inputs.float()
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if batch_idx % 300 == 299:
            print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx+1, running_loss/300))
            running_loss = 0.0       

In [11]:
for epoch in range(100):
    train(epoch)

[1,   300] loss: 1.552
[1,   600] loss: 0.884
[1,   900] loss: 0.777
[2,   300] loss: 0.739
[2,   600] loss: 0.703
[2,   900] loss: 0.705
[3,   300] loss: 0.673
[3,   600] loss: 0.676
[3,   900] loss: 0.648
[4,   300] loss: 0.643
[4,   600] loss: 0.629
[4,   900] loss: 0.627
[5,   300] loss: 0.603
[5,   600] loss: 0.618
[5,   900] loss: 0.613
[6,   300] loss: 0.599
[6,   600] loss: 0.598
[6,   900] loss: 0.586
[7,   300] loss: 0.581
[7,   600] loss: 0.574
[7,   900] loss: 0.578
[8,   300] loss: 0.570
[8,   600] loss: 0.571
[8,   900] loss: 0.558
[9,   300] loss: 0.562
[9,   600] loss: 0.555
[9,   900] loss: 0.551
[10,   300] loss: 0.545
[10,   600] loss: 0.561
[10,   900] loss: 0.537
[11,   300] loss: 0.545
[11,   600] loss: 0.531
[11,   900] loss: 0.547
[12,   300] loss: 0.527
[12,   600] loss: 0.547
[12,   900] loss: 0.532
[13,   300] loss: 0.521
[13,   600] loss: 0.531
[13,   900] loss: 0.532
[14,   300] loss: 0.510
[14,   600] loss: 0.529
[14,   900] loss: 0.529
[15,   300] loss: 0

In [19]:
def predict_save():
    data_test = pd.read_csv("data/otto/test.csv")
    test_inputs = torch.Tensor(np.array(data_test)[:,1:])
    outputs = model.predict(test_inputs.float())
    print(outputs)
    outputs = pd.get_dummies(outputs)
    lables=['Class_1', 'Class_2', 'Class_3', 'Class_4', 'Class_5', 'Class_6', 'Class_7', 'Class_8', 'Class_9']
    outputs.columns = lables
    outputs.insert(0,'id',data_test['id'])
    outputs = pd.DataFrame(outputs)
    outputs.to_csvs('my_predict.csv', index=False)
    
predict_save()

tensor([6, 5, 6,  ..., 6, 6, 6])


ValueError: Categorical categories must be unique